In [1]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
max_g=20

In [2]:
results=pd.read_csv("results.csv")

In [3]:
results.head()

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Sat,2020-09-12,12:30 (14:30),Fulham,0.2,0–3,1.8,Arsenal,NaN,Craven Cottage,Chris Kavanagh,Match Report,NaN
1,1.0,Sat,2020-09-12,15:00 (17:00),Crystal Palace,0.7,1–0,0.8,Southampton,NaN,Selhurst Park,Jonathan Moss,Match Report,NaN
2,1.0,Sat,2020-09-12,17:30 (19:30),Liverpool,3.3,4–3,0.6,Leeds United,NaN,Anfield,Michael Oliver,Match Report,NaN
3,1.0,Sat,2020-09-12,20:00 (22:00),West Ham,1.1,0–2,1.5,Newcastle Utd,NaN,London Stadium,Stuart Attwell,Match Report,NaN
4,1.0,Sun,2020-09-13,14:00 (16:00),West Brom,0.5,0–3,2.2,Leicester City,NaN,The Hawthorns,Anthony Taylor,Match Report,NaN


In [4]:
results=results[['Home','xG','Score','xG.1','Away']]
results=results[results['xG'].notna()]

In [5]:
results.head()

,Home,xG,Score,xG.1,Away
0,Fulham,0.2,0–3,1.8,Arsenal
1,Crystal Palace,0.7,1–0,0.8,Southampton
2,Liverpool,3.3,4–3,0.6,Leeds United
3,West Ham,1.1,0–2,1.5,Newcastle Utd
4,West Brom,0.5,0–3,2.2,Leicester City


In [6]:
xpoints = {team:0 for team in results['Home'].unique()}

In [7]:
xpoints

{'Fulham': 0,
 'Crystal Palace': 0,
 'Liverpool': 0,
 'West Ham': 0,
 'West Brom': 0,
 'Tottenham': 0,
 'Sheffield Utd': 0,
 'Brighton': 0,
 'Everton': 0,
 'Leeds United': 0,
 'Manchester Utd': 0,
 'Arsenal': 0,
 'Southampton': 0,
 'Newcastle Utd': 0,
 'Chelsea': 0,
 'Leicester City': 0,
 'Aston Villa': 0,
 'Wolves': 0,
 'Burnley': 0,
 'Manchester City': 0}

In [8]:
probs=np.matmul(np.array([poisson.pmf(i,0.93) for i in range(max_g)]).reshape(max_g,1),np.array([poisson.pmf(i,2.01) for i in range(max_g)]).reshape(1,max_g))

In [9]:
np.triu(probs).sum()-np.trace(probs)

0.6246167695715348

In [10]:
np.tril(probs).sum()-np.trace(probs)

0.16671269794768587

In [11]:
np.trace(probs)

0.20867053248070885

In [12]:
for index,match in results.iterrows():
    homeG = float(match['xG'])
    awayG = float(match['xG.1'])
    probs = np.matmul(np.array([poisson.pmf(i,homeG) for i in range(max_g)]).reshape(max_g,1),np.array([poisson.pmf(i,awayG) for i in range(max_g)]).reshape(1,max_g))
    homeWP = np.tril(probs).sum()-np.trace(probs)
    awayWP = np.triu(probs).sum()-np.trace(probs)
    drawP = np.trace(probs)
    homeP = homeWP*3+drawP*1
    awayP = awayWP*3+drawP*1
    xpoints[match['Home']]+=homeP
    xpoints[match['Away']]+=awayP
    

In [13]:
xpoints

{'Fulham': 29.62601060236442,
 'Crystal Palace': 25.003774186473034,
 'Liverpool': 42.73869017313096,
 'West Ham': 36.628608366548605,
 'West Brom': 17.45885667459625,
 'Tottenham': 31.394298116570596,
 'Sheffield Utd': 23.29530946656363,
 'Brighton': 37.90160971984598,
 'Everton': 29.193651667381168,
 'Leeds United': 30.55493772137935,
 'Manchester Utd': 38.80002954669131,
 'Arsenal': 35.87012478873377,
 'Southampton': 27.687059599601348,
 'Newcastle Utd': 25.604032617763284,
 'Chelsea': 43.10471550891694,
 'Leicester City': 38.121895132198084,
 'Aston Villa': 34.272752491394854,
 'Wolves': 31.434866056004626,
 'Burnley': 24.89883414997197,
 'Manchester City': 50.34662735546532}

In [14]:
for team in sorted(xpoints, key=xpoints.get,reverse=True):
    print('{}: {}'.format(team,round(xpoints[team])))

Manchester City: 50
Chelsea: 43
Liverpool: 43
Manchester Utd: 39
Leicester City: 38
Brighton: 38
West Ham: 37
Arsenal: 36
Aston Villa: 34
Wolves: 31
Tottenham: 31
Leeds United: 31
Fulham: 30
Everton: 29
Southampton: 28
Newcastle Utd: 26
Crystal Palace: 25
Burnley: 25
Sheffield Utd: 23
West Brom: 17


In [15]:
sum(xpoints.values())

653.9366839415956

In [16]:
pltable = pd.read_csv('pltable.txt')

In [17]:
pltable['Pts'].sum()

645